<a href="https://colab.research.google.com/github/G0nkly/Ready-Set-Prophecy/blob/main/NLP_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Mini-Datensatz
data = pd.DataFrame({
    "text": [
        "Meine Stromrechnung ist zu hoch",
        "Mein Strom ist ausgefallen",
        "Ich möchte meinen Tarif wechseln",
        "Bitte senden Sie mir die Rechnung",
        "Techniker war nicht da bei Störung",
        "Rechnung fehlerhaft, Betrag prüfen",
        "Leitung abgefallen, Gefahr für Menschen",
        "Kleinere Spannungsschwankung"
    ],
    "label": ["Rechnung","Störung","Tarif","Rechnung","Störung","Rechnung","Störung","Störung"]
})

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.25, random_state=42)

# Vektorisierung + Modell
vec = TfidfVectorizer()
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

print("Baseline TF-IDF + Logistic Regression")
print(classification_report(y_test, y_pred))

Baseline TF-IDF + Logistic Regression
              precision    recall  f1-score   support

    Rechnung       0.00      0.00      0.00         1
     Störung       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Installieren falls nötig: !pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier

# Modell laden (klein für Demo)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Texte in Embeddings umwandeln
X_train_emb = embed_model.encode(X_train)
X_test_emb = embed_model.encode(X_test)

# Klassifikator
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X_train_emb, y_train)
y_pred_emb = clf.predict(X_test_emb)

print("Mittlere Komplexität: Sentence Embeddings + Random Forest")
print(classification_report(y_test, y_pred_emb))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyError: 5

In [ ]:
# Installieren falls nötig:
# !pip install transformers datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.preprocessing import LabelEncoder

# Labels in Zahlen
le = LabelEncoder()
data["label_id"] = le.fit_transform(data["label"])

# Train/Test Dataset
train_df = data.sample(frac=0.75, random_state=42)
test_df = data.drop(train_df.index)

train_ds = Dataset.from_pandas(train_df[["text","label_id"]])
test_ds = Dataset.from_pandas(test_df[["text","label_id"]])

# Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(le.classes_))

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,       # für Demo nur 1 Epoch
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=2,
    logging_dir='./logs',
    do_train=True,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

# Training (läuft 1 Epoch für Demo)
trainer.train()

# Evaluation
eval_results = trainer.evaluate()
print("DistilBERT Fine-Tuning Evaluation:")
print(eval_results)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.weight | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but n

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.